In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn import metrics
import cvxpy as cp
import os
import sys
import time


In [26]:
class simplexAlgorithm():
    def __init__(self, A, b, c, z):
        self.A = np.array(A)
        self.b = np.array(b)
        self.c = np.array(c)
        self.m = len(b)
        self.n = len(c)
        self.z = np.array(z)
        
    def find_tight_rows(self):
        """
        Finds the tight constraints at point z
        """
        val = np.isclose(self.A @ self.z, self.b)
        return val
    
    def compute_direction(self, A_tight):
        """
        Computes the direction to move from current point
        """
        A_inv = np.linalg.inv(A_tight)
        direction = A_inv @ self.c
        return -direction
    
    def solve(self, iterate=1000):
        iterations = iterate
        while iterations >= 0:
            tight_rows = self.find_tight_rows()
            A_tight = self.A[tight_rows]
            
            # direction to move
            direction = self.compute_direction(A_tight)
            
            if np.all(direction >= 0):
                print("Optimal solution found")
                break
            
            non_tight_rows = np.logical_not(tight_rows)
            
            # coeff = (self.b[non_tight_rows] - self.A[non_tight_rows] @ self.z) / (self.A[non_tight_rows] @ direction)
            coeff = np.where(self.A[non_tight_rows] @ direction != 0, (self.b[non_tight_rows] - self.A[non_tight_rows] @ self.z) / (self.A[non_tight_rows] @ direction), np.inf)
            coeff[coeff < 0] = np.inf
            
            # update z
            self.z += np.min(coeff) * direction
            iterations -= 1
            
        return self.z
    

# size m x n
m = 3
n = 4

# generate matrix of size (m+2) x (n+1)
input_mat = np.random.randint(1, 10, size=(m+2, n+1))

# c is the first row till n columns
c = input_mat[0, :n]

# z is the second row till n columns
z = input_mat[1, :n]

# b is the last column from 3rd row till (m+2)th row
b = input_mat[2:, n]

# A is the m x n matrix from 3rd row till (m+2)th row and first n columns
A = input_mat[2:, :n]

# c = np.array([3, 2])
# z = np.array([1, 2])
# b = np.array([3, 4])
# A = np.array([[1, 1], [2, 1]])

c = np.array([9, 2])
z = np.array([0.7,0])
b = np.array([3, 6, 9])
A = np.array([[5, 2], [6, 2], [3, 8]])

# print rank of A
print("Rank of A: ", np.linalg.matrix_rank(A))
print(A.shape)

# perform simplex algorithm  and get the optimal solution and objective value
simplex = simplexAlgorithm(A, b, c, z)
optimal_solution = simplex.solve()
objective_value = c @ optimal_solution

print("Optimal solution: ", optimal_solution)
print("Objective value: ", objective_value)


        

Rank of A:  2
(3, 2)


LinAlgError: Last 2 dimensions of the array must be square

In [30]:
import numpy as np

class simplexAlgorithm():
    def __init__(self, A, b, c, z):
        self.A = np.array(A)
        self.b = np.array(b)
        self.c = np.array(c)
        self.z = np.array(z)
        
    def find_tight_rows(self):
        """
        Finds the tight constraints at point z
        """
        val = np.isclose(self.A @ self.z, self.b)
        return val
    
    def compute_direction(self, A_tight):
        """
        Computes the direction to move from current point
        """
        try:
            A_inv = np.linalg.inv(A_tight)
        except np.linalg.LinAlgError:
            return None
        
        direction = A_inv @ self.c
        return -direction
    
    def solve(self, iterate=1000):
        iterations = iterate
        while iterations >= 0:
            tight_rows = self.find_tight_rows()
            A_tight = self.A[tight_rows]
            
            # Ensure that A_tight is square and of full rank
            if A_tight.shape[0] != A_tight.shape[1] or np.linalg.matrix_rank(A_tight) < A_tight.shape[1]:
                print("Insufficient or excess tight constraints. Exiting early.")
                return self.z
            
            # direction to move
            direction = self.compute_direction(A_tight)
            
            # If direction is None, matrix inversion failed
            if direction is None:
                print("Failed to compute direction. Exiting early.")
                return self.z
            
            if np.all(direction >= 0):
                print("Optimal solution found")
                break
            
            non_tight_rows = np.logical_not(tight_rows)
            coeff = np.where(self.A[non_tight_rows] @ direction != 0, 
                             (self.b[non_tight_rows] - self.A[non_tight_rows] @ self.z) / (self.A[non_tight_rows] @ direction), 
                             np.inf)
            coeff[coeff < 0] = np.inf
            
            # update z
            self.z += np.min(coeff) * direction
            iterations -= 1
            
        return self.z

# Initialize variables
c = np.array([9, 2])
z = np.array([0.9,0])
b = np.array([3, 6, 9])
A = np.array([[5, 2], [6, 2], [3, 8]])

# perform simplex algorithm and get the optimal solution and objective value
simplex = simplexAlgorithm(A, b, c, z)
optimal_solution = simplex.solve()
objective_value = c @ optimal_solution

print("Optimal solution:", optimal_solution)
print("Objective value:", objective_value)


Insufficient or excess tight constraints. Exiting early.
Optimal solution: [0.9 0. ]
Objective value: 8.1


In [ ]:
import numpy as np

def find_tight_rows(A, X, B):
    """
    Determine the tight constraints at point X.
    """
    return np.isclose(np.dot(A, X), B)

def compute_direction(A_tight, C):
    """
    Compute the direction to move from the current point.
    """
    A_inv = np.linalg.pinv(A_tight.T)
    direction = -np.dot(A_inv, C)
    return direction

def simplex_method(A, B, C, X):
    """
    Simplified Simplex Algorithm based on provided assumptions.
    """
    iterations = 0
    while iterations < 1000:  # safety limit
        tight_mask = find_tight_rows(A, X, B)
        A_tight = A[tight_mask]
        
        # Calculate the direction to move in
        v = compute_direction(A_tight, C)

        # If no further improvement is possible, break
        if np.all(v >= 0):
            break

        # Compute step size to the next vertex
        non_tight_mask = np.logical_not(tight_mask)
        ratios = (B[non_tight_mask] - np.dot(A[non_tight_mask], X)) / np.dot(A[non_tight_mask], v)
        positive_ratios = ratios[ratios > 0]

        # Update point
        X += np.min(positive_ratios) * v
        iterations += 1

    return X

def main():
    # Input handling
    C = np.array(list(map(float, input("Enter coefficients of C separated by spaces: ").split())))
    B = np.array(list(map(float, input("Enter B values separated by spaces: ").split())))
    A = []
    print("Enter the A matrix rows:")
    for _ in range(len(B)):
        row = list(map(float, input().split()))
        A.append(row)
    A = np.array(A)
    
    X_init = np.array(list(map(float, input("Enter Initial Feasible Point separated by spaces: ").split())))

    # Run the algorithm
    optimal_X = simplex_method(A, B, C, X_init)
    print(f"The optimal solution is: {optimal_X}")
    print(f"Objective value at optimal solution: {np.dot(C, optimal_X)}")

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np

EPS = 1e-6

def get_direction_vector(A, B, C, X):
    """
    Get the direction vector for X to navigate to the `nearest` neighbor.

    :return: direction vector for X
    """
    # Find the rows of A that are "tight" at the current point X
    equality_indices = np.where(np.abs(np.dot(A, X) - B) < EPS)[0]
    A_tight = A[equality_indices]

    # Compute the direction vectors
    A_inv = np.linalg.inv(np.transpose(A_tight))
    alphas = np.dot(A_inv, C)

    # Identify the first direction with a negative coefficient
    neg_indices = np.where(alphas < 0)[0]
    
    # If there are no negative alphas, the current point is optimal
    if len(neg_indices) == 0:
        return None
    else:
        return -A_inv[neg_indices[0]]

def simplex_algorithm(A, B, C, X):
    """
    Find the optimal solution using the Simplex Algorithm.
    """
    while True:
        # Get the direction vector
        v = get_direction_vector(A, B, C, X)

        # If no improving direction can be found, the current point is optimal
        if v is None:
            return X

        # Otherwise, compute the maximum step size to the next vertex
        non_tight_indices = np.setdiff1d(np.arange(A.shape[0]), np.where(np.abs(np.dot(A, X) - B) < EPS)[0])
        A_non_tight = A[non_tight_indices]
        B_non_tight = B[non_tight_indices]

        t_values = (B_non_tight - np.dot(A_non_tight, X)) / (np.dot(A_non_tight, v) + EPS)
        t_values = t_values[t_values > 0]

        # Move to the next vertex
        X = X + np.min(t_values) * v

def main():
    # Taking input
    print('Enter C vector: ')
    C = np.asarray(list(map(float, input().split('\t'))))

    print('Enter B vector: ')
    B = np.asarray(list(map(float, input().split('\t'))))

    print('Enter A matrix: ')
    A = []
    for _ in range(len(B)):
        A.append(np.asarray(list(map(float, input().split('\t')))))
    A = np.asarray(A)

    print('Enter Initial Feasible Point: ')
    X = np.asarray(list(map(float, input().split('\t'))))

    # Run the Simplex Algorithm
    X_optimal = simplex_algorithm(A, B, C, X)
    print('The Optimal Solution is:', X_optimal)
    print('The Maximum Value is:', np.dot(C, X_optimal))

if __name__ == "__main__":
    main()
